# SENG 550 Project
__Sentiment Analysis using TF-IDF and Random Forests to predict Amazon review ratings using review text and review summary__

Import all libraries

In [0]:
import re
import pandas as pd
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.mllib.tree import RandomForest
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint

Read json data file and convert to dataframe

In [0]:
rdd = spark.read.json('/FileStore/tables/All_Beauty_5.json').rdd
print(rdd.take(1)[0])

Row(asin='B0000530HU', image=None, overall=5.0, reviewText='As advertised. Reasonably priced', reviewTime='09 1, 2016', reviewerID='A3CIUOJXQ5VDQ2', reviewerName='Shelly F', style=Row(Color:=None, Design:=None, Flavor:=' Classic Ice Blue', Scent Name:=None, Size:=' 7.0 oz', Style Name:=None), summary='Five Stars', unixReviewTime=1472688000, verified=True, vote=None)


Create function to parse a string and extract the data we need (overall, reviewText, summary)

In [0]:
def extractFeatures(row):
    """Extracts the features we want from the data.

    Args:
        line (pyspark.sql.types.Row): RDD row.

    Returns:
        tuple: A tuple with overall score, reviewText, and summary.
    """    
    overall = float(row.asDict()['overall'])
    
    reviewText = row.asDict()['reviewText']
    if reviewText:
        reviewText = re.sub(r'http\S+', '', reviewText)
        reviewText = re.sub(r'[^\w\s]', '', reviewText)
        reviewText = reviewText.lower()
        
    summary = row.asDict()['summary']
    if summary:
        summary = re.sub(r'http\S+', '', summary)
        summary = re.sub(r'[^\w\s]', '', summary)
        summary = summary.lower()
        
    processedText = ''
    if reviewText and summary:
        processedText = summary + ' ' + reviewText
        
    return (overall, processedText)

Apply `extractFeatures` to base RDD

In [0]:
extractedRDD = rdd.map(extractFeatures).collect()
print(extractedRDD)

[(5.0, 'five stars as advertised reasonably priced'), (5.0, 'good for the face like the oder and the feel when i put it on my face  i have tried other brands but the reviews from people i know they prefer the oder of this brand not hard on the face when dry  does not leave dry skin'), (1.0, 'smells awful i bought this to smell nice after i shave  when i put it on i smelled awful  i am 19 and i smelled like a grandmother with too much perfume'), (5.0, 'truth is there is nothing like an aqua velva man hey i am an aqua velva man and absolutely love this stuff been using it for over 50 years this is a true after shave lotion classic not quite sure how many women that have been attracted to me because of aqua velva  i do know for sure that its just to many to count ha  not sure how long this has been around but the williams company ran a paper advertisement taken from a 1949 magazine which features ralph bellamy of detective story and ezio pinza of south pacific for aqua velva after shave l

Text processing to remove punctuation and convert all characters to lowercase

In [0]:
df = spark.createDataFrame(extractedRDD, ['overall', 'review'])

tokenizer = Tokenizer(inputCol="review", outputCol="tokenizedReview")
tk = tokenizer.transform(df)

hashingTF = HashingTF(inputCol="tokenizedReview", outputCol="rawTokenizedReview", numFeatures=100)
tf = hashingTF.transform(tk)

tf.cache()
idf = IDF(inputCol="rawTokenizedReview", outputCol="features").fit(tf)
tfidf = idf.transform(tf)

df = tfidf.select("overall", "features").rdd

Perform Random Forest Classification

In [0]:
labelledPointRDD = df.map(lambda x: LabeledPoint(x['overall'], x['features'].toArray()))

training, testing = labelledPointRDD.randomSplit([0.7, 0.3])

rf = RandomForest.trainClassifier(sc.parallelize(training.collect()), 6, {}, numTrees=5, maxDepth=7, seed=0)

X_test = testing.map(lambda x: x.features)
y_test = testing.map(lambda x: x.label)

y_pred = rf.predict(X_test)

both = list(zip(y_test.collect(),y_pred.collect()))

acc = sum(1 for test, pred in both if test == pred) / float(test_data.count())
print("Model accuracy: %.2f%%" % (acc * 100))

Model accuracy: 87.52%
